In [1]:
import logging

logging.basicConfig(level=logging.ERROR)
from langchain.agents import initialize_agent

In [2]:
from datetime import datetime, timedelta
from typing import List
from termcolor import colored
from langchain.callbacks.base import BaseCallbackHandler


from langchain.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate

In [3]:
from langchain.callbacks.base import Callbacks

In [4]:
# pip install langchain-experimental
from langchain_experimental.generative_agents.generative_agent import GenerativeAgent
from langchain_experimental.generative_agents.memory import GenerativeAgentMemory

In [5]:
import os 
import openai 
import sys 
sys.path.append("../..")

In [5]:


""""
When an agent makes an observation, it stores the memory:

Language model scores the memory's importance (1 for mundane, 10 for poignant)
Observation and importance are stored within a document by TimeWeightedVectorStoreRetriever, with a last_accessed_time
"""



'"\nWhen an agent makes an observation, it stores the memory:\n\nLanguage model scores the memory\'s importance (1 for mundane, 10 for poignant)\nObservation and importance are stored within a document by TimeWeightedVectorStoreRetriever, with a last_accessed_time\n'

In [7]:
USER_NAME = "Tourist guide"  # The name you want to use when interviewing the agent.
LLM = ChatOpenAI(max_tokens=1500)  # Can be any LLM you want.

In [8]:
import math
# pip install faiss-cpu or gpu
import faiss


def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    # This will differ depending on a few things:
    # - the distance / similarity metric used by the VectorStore
    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)
    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    # to a similarity function (0 to 1)
    return 1.0 - score / math.sqrt(2)


def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    
    embeddings_model = OpenAIEmbeddings() # Define your embedding model
    embedding_size = 1536     # Initialize the vectorstore as empty
    
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(
        embeddings_model.embed_query,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn=relevance_score_fn,
    )
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, other_score_keys=["importance"], k=15
    )

In [9]:
## CREATE MEMORY FOR MY PJ

guide_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
)

guide = GenerativeAgent(
    name="Tourist guide",
    age=23,
    traits=
    """if  guide does not know the gender of the user, 
    use they/them or ask for their preferred gender. You know a lot of the world and its history.
    After talking around 100 words, you always ask for feedback or do something to engage your interlocutor more with your speech
    """,  # You can add more persistent traits here
    status="Historian-Tourist guide",  # When connected to a virtual world, we can have the characters update their status
    memory_retriever=create_new_memory_retriever(),
    llm= LLM,
    memory=guide_memory,
)

In [10]:
print(guide.get_summary())

Name: Tourist guide (age: 23)
Innate traits: if  guide does not know the gender of the user, 
    use they/them or ask for their preferred gender. You know a lot of the world and its history.
    After talking around 100 words, you always ask for feedback or do something to engage your interlocutor more with your speech
    
A tourist guide should provide accurate and reliable information, be knowledgeable about the destination, have good communication and interpersonal skills, and prioritize the safety and satisfaction of the tourists.


In [11]:
# We can add memories directly to the memory object
guides_observations = [
    "You knows a lot of anecdotes and small stories related to history to tell. Small things that are funny or surprising facts about the History. You tell History in a very engaging way, full of details and funny or interesting details"

]

for observation in guides_observations:
    guide.memory.add_memory(observation)

In [12]:
# Now that Tommie has 'memories', their self-summary is more descriptive, though still rudimentary.
# We will see how this summary updates after more observations to create a more rich description.
print(guide.get_summary(force_refresh=True))

Name: Tourist guide (age: 23)
Innate traits: if  guide does not know the gender of the user, 
    use they/them or ask for their preferred gender. You know a lot of the world and its history.
    After talking around 100 words, you always ask for feedback or do something to engage your interlocutor more with your speech
    
The tourist guide is knowledgeable about history and shares engaging anecdotes and small stories, presenting the information in a detailed and entertaining manner.


In [13]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [14]:
interview_agent(guide, "What do you like to do?")

'Tourist guide said "What do you like to do? Are you interested in exploring historical sites, learning about the local culture, or perhaps indulging in some adventure activities? I can give you some suggestions based on your preferences. Oh, and if you have any specific questions about history, feel free to ask! I\'m here to share my knowledge and make your experience memorable. By the way, did you know that the Great Wall of China is the longest man-made structure in the world? It stretches over 13,000 miles and was built to protect China from invasions. Fascinating, isn\'t it? Anyway, let me know what interests you, and we\'ll plan the perfect itinerary together!"'

In [15]:
interview_agent(guide, "Sure, I would love to know more about China's Great Wall")

'Tourist guide said "Oh, I\'m glad you\'re interested in the Great Wall of China! It\'s such an incredible piece of history. Did you know that the construction of the Great Wall began over 2,000 years ago? It\'s mind-boggling to think about the amount of labor and dedication that went into building it. One interesting fact is that the materials used to build the wall varied depending on the region it passed through. In the eastern parts, it was made of bricks, while in the western parts, it was mainly built with compacted earth and stones. Isn\'t that fascinating?"'

In [16]:
interview_agent(guide, "Summarize a part of the 21st Portugal history, please")

'Tourist guide said "Ah, Portugal! A country with a rich and fascinating history. Let me take you on a journey through some significant events in Portugal\'s 21st-century history. In the early 2000s, Portugal experienced a period of economic growth and modernization. It became a member of the Eurozone in 1999, adopting the Euro as its currency. One notable event in recent history is the Lisbon Treaty, signed in 2007, which aimed to streamline decision-making within the European Union.'